# Quantum `conda` environment

Let's create a new `conda` environment called "q-env" and running Python 3.11:

```bash
conda create -n q-env python=3.11 -y
```

After the activation

```bash
conda activate q-env
```

start by upgrading `pip`, `ipykernel`, and `numpy` via:

```bash
pip install --upgrade pip ipykernel numpy==1.26.4
```

Continue by installing TensorFlow 2.16 (with CUDA 12.3 and cuDNN 8.9):

```bash
pip install tensorflow[and-cuda]==2.16.*
```

The next step is to configure the system paths. You can do it with the following command every time you start a new terminal (or Jupyter session) after activating your conda environment:

```bash
NVIDIA_DIR=$(dirname $(dirname $(python -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)")))

export LD_LIBRARY_PATH=${CONDA_PREFIX}/lib/:${LD_LIBRARY_PATH}
for dir in $(ls -1d $NVIDIA_DIR/*/); do
    if [ -d "${dir}lib" ]; then
        export LD_LIBRARY_PATH="${dir}lib:$LD_LIBRARY_PATH"
        if [[ $(basename $dir) == 'cuda_nvcc' ]] ; then
            export PATH="${dir}bin:$PATH"
        fi
    fi
done

export XLA_FLAGS=--xla_gpu_cuda_data_dir=${CONDA_PREFIX}/lib
```

For your convenience, it is recommended that you automate it by adding the previous lines in the `env_vars.sh` file as described in the following:

```bash
mkdir -p $CONDA_PREFIX/etc/conda/activate.d
vim $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh
```

Test that the installation succeeds and that TensorFlow correctly accesses the GPU by executing what follows:

In [1]:
import tensorflow as tf
import keras as k

print(f"[DEBUG] TensorFlow version {tf.__version__}")
print(f"[DEBUG] Keras version {k.__version__}")

devices = tf.config.list_physical_devices("GPU")
if len(devices) > 0:
    rnd = tf.random.uniform(shape=(100, 1))
    print("[STATUS] GPU available")
else:
    print("[STATUS] GPU not available") 

2024-06-25 08:19:41.230856: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 08:19:43.334133: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[DEBUG] TensorFlow version 2.16.1
[DEBUG] Keras version 3.3.3


2024-06-25 08:19:46.380799: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-25 08:19:46.452155: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-25 08:19:46.454545: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

[STATUS] GPU available


Continue by installing Jax based on CUDA 12.3 (locally preinstalled):

```bash
pip install --upgrade "jax[cuda12_local]==0.4.28" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
```

and Flax (via `pip` channel):

```bash
pip install flax
```

Test that the installation succeeds and that Jax correctly accesses the GPU by executing what follows:

In [2]:
import jax
import flax

print(f"[DEBUG] Jax version {jax.__version__}")
print(f"[DEBUG] Flax version {flax.__version__}")

def jax_has_gpu():
    try:
        _ = jax.device_put(jax.numpy.ones(1), device=jax.devices('gpu')[0])
        return True
    except:
        return False

if jax_has_gpu():
    print("[STATUS] GPU available")
else:
    print("[STATUS] GPU not available")

[DEBUG] Jax version 0.4.28
[DEBUG] Flax version 0.8.4
[STATUS] GPU available


Continue by installing Pennylane Lightning with GPU-acceleration:

```bash
pip install pennylane pennylane-lightning-gpu
```

To install Lightning with NVIDIA CUDA support, the following packages need to be installed via `pip`:

```bash
pip install nvidia-cusparse-cu12 nvidia-cublas-cu12 nvidia-cuda-runtime-cu12 custatevec-cu12
```

<!--
Lightning-GPU requires also the cuQuantum SDK that can be installed by using `conda`:

```bash
conda install -c conda-forge cuquantum-python cuda-version=12.3 -y
```

within the Python environment site-packages directory using pip or conda or the SDK library path appended to the LD_LIBRARY_PATH environment variable. Please see the cuQuantum SDK install guide for more information.

```bash
echo 'export CUQUANTUM_ROOT=${CONDA_PREFIX}' >> $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh

source $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh
```
-->

Test that the installation succeeds and that Pennylane Lightning correctly accesses the GPU by executing what follows:

In [3]:
import pennylane as qml

print(f"[DEBUG] PennyLane version {qml.__version__}")

dev = qml.device("lightning.gpu", wires=2)

@qml.qnode(dev)
def circuit():
  qml.Hadamard(wires=0)
  qml.CNOT(wires=[0,1])
  return qml.expval(qml.PauliZ(0))

try:
    print("[DEBUG] Circuit:", circuit())
    # Circuit: 0.0
    print("[STATUS] GPU available")
except:
    print("[STATUS] GPU not available")

[DEBUG] PennyLane version 0.36.0


LightningException: [/project/pennylane_lightning/core/src/simulators/lightning_gpu/utils/cuStateVec_helpers.hpp][Line:101][Method:make_shared_cusv_handle]: Error in PennyLane Lightning: custatevec memory allocation failed

Continue by installing Qiskit Aer with GPU-acceleration (based on CUDA 12):

```bash
pip install qiskit qiskit-aer-gpu
```

Test that the installation succeeds and that Qiskit Aer correctly accesses the GPU by executing what follows:

In [4]:
import qiskit
from qiskit_aer import AerSimulator

print(f"[DEBUG] Qiskit version {qiskit.__version__}")

# Generate 3-qubit GHZ state
circ = qiskit.QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(1, 2)
circ.measure_all()

# Construct an ideal simulator
aersim = AerSimulator()

# Perform an ideal simulation
try:
    result_ideal = aersim.run(circ).result()
    counts_ideal = result_ideal.get_counts(0)
    print('[DEBUG] Counts(ideal):', counts_ideal)
    # Counts(ideal): {'000': 493, '111': 531}
    print("[STATUS] GPU available")
except:
    print("[STATUS] GPU not available")

[DEBUG] Qiskit version 1.1.0
[DEBUG] Counts(ideal): {'111': 497, '000': 527}
[STATUS] GPU available


Finally, continue by installing Qibo, QiboJIT, and CuPy via `conda`:

```bash
conda install qibo qibojit -y
conda install -c conda-forge cupy cuda-version=12.3 -y
```

In [5]:
import cupy as cp

print(f"[DEBUG] CuPy version {cp.__version__}")

n_cuda_devices = cp.cuda.runtime.getDeviceCount()

if n_cuda_devices > 0:
    x = cp.array([1, 2, 3])
    x.device
    print("[STATUS] GPU available")
else:
    print("[STATUS] GPU not available")

/envs/q-jax-tf216/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[DEBUG] CuPy version 13.2.0
[STATUS] GPU available
